In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [3]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [4]:
X_new_scaled = X_test_scaled

In [5]:
class MySGDOptimizer(keras.optimizers.Optimizer):
    def __init__(self, learning_rate=0.001, momentum=0.9, name="MySGDOptimizer", **kwargs):
        """Call super().__init__() and use _set_hyper() to store hyperparameters"""
        super().__init__(name, **kwargs)
        self._set_hyper("learning_rate", kwargs.get("lr", learning_rate)) # handle lr=learning_rate
        self._set_hyper("decay", self._initial_decay) # 
        self._set_hyper("momentum", momentum)
    
    def _create_slots(self, var_list):
        """For each model variable, create the optimizer variable associated with it.
        TensorFlow calls these optimizer variables "slots".
        For momentum optimization, we need one momentum slot per model variable.
        """
        for var in var_list:
            self.add_slot(var, "momentum")

    @tf.function
    def _resource_apply_dense(self, grad, var):
        """Update the slots and perform one optimization step for one model variable
        """
        var_dtype = var.dtype.base_dtype
        lr_t = self._decayed_lr(var_dtype) # handle learning rate decay
#         momentum_var = self.get_slot(var, "momentum")
#         momentum_hyper = self._get_hyper("momentum", var_dtype)
#         momentum_var.assign(momentum_var * momentum_hyper - (1. - momentum_hyper)* grad)
        var.assign_sub(grad * lr_t)

    def _resource_apply_sparse(self, grad, var):
        raise NotImplementedError

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            "learning_rate": self._serialize_hyperparameter("learning_rate"),
            "decay": self._serialize_hyperparameter("decay"),
            "momentum": self._serialize_hyperparameter("momentum"),
        }

In [6]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [7]:
model = keras.models.Sequential([keras.layers.Dense(1, input_shape=[8])])
model.compile(loss="mse", optimizer=MySGDOptimizer(learning_rate=0.001))
model.fit(X_train_scaled, y_train, epochs=20)

2021-08-23 01:35:00.314081: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-23 01:35:02.147425: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20
363/363 [==============================] - 1s 439us/step - loss: 3.7378
Epoch 2/20
363/363 [==============================] - 0s 522us/step - loss: 1.4864
Epoch 3/20
363/363 [==============================] - 0s 550us/step - loss: 0.9196
Epoch 4/20
363/363 [==============================] - 0s 779us/step - loss: 0.7603
Epoch 5/20
363/363 [==============================] - 0s 685us/step - loss: 0.7058
Epoch 6/20
363/363 [==============================] - 0s 628us/step - loss: 0.6775
Epoch 7/20
363/363 [==============================] - 0s 650us/step - loss: 0.6608
Epoch 8/20
363/363 [==============================] - 0s 523us/step - loss: 0.6483
Epoch 9/20
363/363 [==============================] - 0s 519us/step - loss: 0.6354
Epoch 10/20
363/363 [==============================] - 0s 518us/step - loss: 0.6241
Epoch 11/20
363/363 [==============================] - 0s 558us/step - loss: 0.6166
Epoch 12/20
363/363 [==============================] - 0s 700us/step - loss: 0.6061
E

In [10]:
class SGOptimizer(keras.optimizers.Optimizer):
    def __init__(self, learning_rate=0.01, name="SGOptimizer", **kwargs):
        """Call super().__init__() and use _set_hyper() to store hyperparameters"""
        super().__init__(name, **kwargs)
        self._set_hyper("learning_rate", kwargs.get("lr", learning_rate)) # handle lr=learning_rate
        self._is_first = True
    
    def _create_slots(self, var_list):
        """For each model variable, create the optimizer variable associated with it.
        TensorFlow calls these optimizer variables "slots".
        For momentum optimization, we need one momentum slot per model variable.
        """
        for var in var_list:
            self.add_slot(var, "pv") #previous variable i.e. weight or bias
        for var in var_list:
            self.add_slot(var, "pg") #previous gradient


    @tf.function
    def _resource_apply_dense(self, grad, var):
        """Update the slots and perform one optimization step for one model variable
        """
        var_dtype = var.dtype.base_dtype
        lr_t = self._decayed_lr(var_dtype) # handle learning rate decay
        new_var_m = var - grad * lr_t
        pv_var = self.get_slot(var, "pv")
        pg_var = self.get_slot(var, "pg")
        
        if self._is_first:
            self._is_first = False
            new_var = new_var_m
        else:
            cond = grad*pg_var >= 0
            print(cond)
            avg_weights = (pv_var + var)/2.0
            new_var = tf.where(cond, new_var_m, avg_weights)
        pv_var.assign(var)
        pg_var.assign(grad)
        var.assign(new_var)

    def _resource_apply_sparse(self, grad, var):
        raise NotImplementedError

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            "learning_rate": self._serialize_hyperparameter("learning_rate"),
        }

In [11]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [12]:
model = keras.models.Sequential([keras.layers.Dense(1, input_shape=[8])])
model.compile(loss="mse", optimizer=SGOptimizer(learning_rate=0.001))
model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
Tensor("GreaterEqual:0", shape=(1,), dtype=bool, device=/job:localhost/replica:0/task:0/device:CPU:0)
363/363 [==============================] - 1s 572us/step - loss: 3.7378
Epoch 2/50
363/363 [==============================] - 0s 650us/step - loss: 1.4864
Epoch 3/50
363/363 [==============================] - 0s 588us/step - loss: 0.9201
Epoch 4/50
363/363 [==============================] - 0s 676us/step - loss: 0.7626
Epoch 5/50
363/363 [==============================] - 0s 567us/step - loss: 0.7082
Epoch 6/50
363/363 [==============================] - 0s 630us/step - loss: 0.6792
Epoch 7/50
363/363 [==============================] - 0s 669us/step - loss: 0.6616
Epoch 8/50
363/363 [==============================] - 0s 621us/step - loss: 0.6487
Epoch 9/50
363/363 [==============================] - 0s 691us/step - loss: 0.6355
Epoch 10/50
363/363 [==============================] - 0s 657us/step - loss: 0.6242
Epoch 11/50
363/363 [==============================] - 0s 581us/ste

In [13]:
a = tf.Variable([-1,2,2,0])
b = tf.Variable([5,6,7,8])
cond = a*b >= 0
cond

<tf.Tensor: shape=(4,), dtype=bool, numpy=array([False,  True,  True,  True])>

In [14]:

tf.where(cond, a, b)

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([5, 2, 2, 0], dtype=int32)>